In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
#import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interactive
from ipywidgets import Layout, Button, Label
from ipywidgets import GridspecLayout,HBox, VBox, Box


In [ ]:
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots

import bqplot as bq

In [ ]:
all_smooth=list(np.arange(0,0.3,0.05))
all_smooth=[round(x,2) for x in all_smooth]

In [ ]:
all_index=['EPU', 'IBC_BR', 'ICE', 'IE_E', 'IPCA', 'ISA_E', 'selic']

In [ ]:
dfs=[]
names=[]
names_url=[]
url='https://raw.githubusercontent.com/LeonardoGiovanelli/TCC_INSPER_LCA/main/'

fix='datas_dash_'

for i in all_index:
    name_aux=[]
    names_aux_url=[]
    datas=[]
    for s in all_smooth:
        aux=fix+i+'_'+str(s)+'.csv'
        name_aux.append(aux)
        
        aux_url=url+aux
        names_aux_url.append(aux_url)
        data=pd.read_csv(aux_url)
        datas.append(data)
    names.append(name_aux) 
    names_url.append(names_aux_url)
    dfs.append(datas)

# Previsão de indíces econômicos

In [ ]:
def plot_index_ploty(indice,smooth):
    nc=str(indice)+'_pred'
    n_indice=all_index.index(indice)
    n_smooth=all_smooth.index(smooth)
    r=dfs[n_indice][n_smooth]
    
    r['Dia']="1"
    r['data']=pd.to_datetime(r["Ano"].astype(str) + "/" + r["Mes"].astype(str) + "/" + r["Dia"])
    
    #############
    r_0na=r.dropna()
    
 ######### 0 na #################
  
    
    fig=go.Figure()
    
    fig.update_layout(
    title="Previsto x Testado",
    xaxis_title="Data",
    yaxis_title=indice
    )
        
    fig = fig.add_trace(go.Scatter(
            x=r_0na['data'],
            y=r_0na[nc],
             error_y=dict(
            type='data',
            symmetric=True,
            array=r_0na['Erro']),
             name=nc
        ))
    fig.add_trace(go.Scatter(
            x=r_0na['data'],
            y=r_0na[str(indice)],
               name=str(indice)
    ))
        
    fig.show()   

########################## previsto
    r_pred=r[r.isna().any(axis=1)]
    #r_pred=r_pred.iloc[:,[0,1,-2,-1]]
    
    fig_px = px.bar(r_pred, 
                 x='data', 
                 y=nc, 
                 error_y="Erro",
                 title="Previsão")
    fig_px.show() 

# Erro na previsão

In [ ]:
url='https://raw.githubusercontent.com/LeonardoGiovanelli/TCC_INSPER_LCA/main/datas_dash_erro_'
dfs_erro=[url+x+'.csv' for x in all_index]
dfs_erro=[pd.read_csv(x) for x in dfs_erro]

In [ ]:
def erro_smooth_indices(smooth,indice):
    nr=int(round(2+3*smooth/.05,0))
    np=int(round(3+3*smooth/.05,0))
    real=dfs_erro[indice].iloc[:,nr]
    previsto=dfs_erro[indice].iloc[:,np]

    erro_div=[p/r-1 for p,r in zip(previsto,real)]
    return(erro_div)

def erro_mensal_plot(smooth):
    erro_mensal=pd.DataFrame([erro_smooth_indices(smooth,i) for i in range(len(all_index))]).transpose()
    erro_mensal.columns=all_index
    return(erro_mensal)

In [ ]:
def erro_indi_smooth(indice,smooth):
    nr=int(round(2+3*smooth/.05,0))
    np=int(round(3+3*smooth/.05,0))
    real=dfs_erro[indice].iloc[:,nr]
    previsto=dfs_erro[indice].iloc[:,np]

    erro_div=[p/r-1 for p,r in zip(previsto,real)]
    erro_div_mean=sum(erro_div)/len(erro_div)
    return(erro_div_mean)

def erro_indi_smooths(indice,smooths):
    erro_div_means=[erro_indi_smooth(indice,s) for s in smooths]
    return(erro_div_means)

In [ ]:
erros=pd.DataFrame([erro_indi_smooths(i,all_smooth) for i  in range(len(all_index))]).transpose()
erros.columns=all_index
erros.index=all_smooth
erros=erros*100
erros=erros.round(1)

In [ ]:
erros_de=erros.describe().round(0)

# Previsão

In [ ]:
def index(indice,slider):
    all_dims=['EPU','IBC_BR','ICE','IE_E','IPCA','ISA_E','selic']
    all_smooth=list(np.arange(0,0.3,0.05))
    all_smooth=[round(x,2) for x in all_smooth]
    n_indice=all_dims.index(indice)
    
    r=dfs_erro[n_indice]
    
    r['Dia']="1"
    r['data']=pd.to_datetime(r["Ano"].astype(str) + "/" + r["Mes"].astype(str) + "/" + r["Dia"])
    r=r.drop(columns=['Ano','Mes','Dia'])
    
    r=r.set_index(['data'])
    n_smooth_0=int(3/0.05*slider)
    n_smooth_1=int(n_smooth_0+3)
    r=r.iloc[:,n_smooth_0:n_smooth_1]
    r=r.reset_index()
    return(r)

In [ ]:
def plot_index_erro_ploty(indice,smooth):
    
 ############ get data   
    r=index(indice,smooth)
    
    n_real=r.columns[1]
    n_pred=r.columns[2]
    n_erro=r.columns[3]
#############  

    fig = make_subplots(rows=1, cols=2)
################ comparação
    fig = fig.add_trace(go.Scatter(
        x=r['data'],
        y=r[n_pred],
         error_y=dict(
        type='data',
        symmetric=True,
        array=r[n_erro]),
         name=n_pred),
           row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=r['data'],
        y=r[n_real],
       name=n_real),
         row=1, col=1)
    fig.update_yaxes( title=indice+' s'+str(smooth),row=1, col=1)

### box
    df=erro_mensal_plot(smooth)
    df=df*100
    df=df.round(0)
    x=df.columns
    y=list(range(len(df)))
    
    for xd, yd in zip(x, y):
            fig.add_trace(
                go.Box(
                y=df.iloc[:,yd],
                name=xd,
                jitter=0.5,
                boxpoints=False,
                marker_color='black')
                ,row=1,col=2)
    fig.update_yaxes( title='Erro '+indice+' s'+str(smooth)+'(%)',row=1, col=2)
####### show
    fig.show()

In [ ]:
def plot_graphs(indice,smooth):
    plot_index_erro_ploty(indice,smooth)
    #plot_index_ploty(indice,smooth)

# PLOTS

In [ ]:
smooth=ipywidgets.FloatSlider(value=0,min=0,max=0.25,step=0.05)

In [ ]:
interactive_plot_graphs = interactive(plot_graphs,
                                     indice=all_index,
                                     smooth=smooth)

output = interactive_plot_graphs.children[-1]

interactive_plot_graphs

In [ ]:
#interactive_erro_mensal_plot = interactive(erro_mensal_plot,
                                        #smooth=smooth)

#output = interactive_erro_mensal_plot.children[-1]

#interactive_erro_mensal_plot